In [1]:
import numpy as np
import pandas as pd
import datetime

from cmf_engine.credentials import *
from cmf_engine.fetch_data import *
from cmf_engine.transform import *
from cmf_engine.infer import *
from cmf_engine.postprocess import *
from cmf_engine.model_selection import *

MODEL_PATH = os.path.abspath("./bin/model/")
# from cmf_engine.fetch_data import *

In [2]:
def main(counter,dff):
    print("---------  Counter: ", counter)
    path = 'C:/Users/bgondaliya/git/cmf-mvp3-quarter/bin/local/'
    
    # 0. get pond Credentials
    cred = credentials(path)
    cxn_ODS,cxn_6438 = cred.read_cred()
    
    # 1. to get the data
    data = TeleportData(cxn_6438,cxn_ODS)
    raw_df = data.get_data()
    oil_df = data.get_eia_data(cxn_ODS)
#     raw_df.to_csv('raw_df.csv')
#     oil_df.to_csv('oil_df.csv')
    
    # 2.define variables
    basins = ['AML', 'ASIA', 'MENA', 'OAT', 'RCA', 'Worldwide']
    
    
    
    
    data_start_date = "2017-01-01"
    pickup_date = pd.to_datetime("2022-02-01",format="%Y-%m-%d")+ relativedelta(months=counter*3)#str(datetime.date(date.today().year, date.today().month, 1))
    quarter = (pickup_date.month - 1) // 3 + 1
    forecast_start_date = pickup_date.strftime('%YQ{}'.format(quarter))
    refresh_date =  pd.to_datetime("2022-02-16",format="%Y-%m-%d")+ relativedelta(months=counter*3) #str(date.today())
    refresh_date = refresh_date.strftime('%YQ{}'.format(quarter))
    duration_of_forecast = 4 #quarter


    print("refresh_date:",refresh_date)
#     data_start_date = "2020-05-01"
#     forecast_start_date = str(pd.to_datetime("2022-03-01",format="%Y-%m-%d")+ relativedelta(months=counter))#str(datetime.date(date.today().year, date.today().month, 1))
#     refresh_date =  str(pd.to_datetime("2022-03-26",format="%Y-%m-%d")+ relativedelta(months=counter)) #str(date.today())
#     duration_of_forecast = 12
    
    
    raw_df = raw_df[raw_df["SL BASIN (CODE)"]!="SL_Elims"]
    raw_df["SL BASIN (CODE)"] = raw_df["SL BASIN (CODE)"]+"-"+raw_df["SL DIVISION (CODE)"]


    # Dep Vars
    rev_var = ["rev"]
    cm_var = ['cm']

    # Baseline Model

    rev_features = ['c4c_rev', 'gac_rev',
           'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 'gac_activity(t-1)',
           'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
           'gac_activity(t+2)',
           'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
           'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
           'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
           'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
           'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)']



    cm_features = [
            'rev', 'c4c_rev', 'gac_rev',
           'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 'gac_activity(t-1)',
           'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
           'gac_activity(t+2)',
           'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
           'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
           'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
           'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
           'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',
           'rev_activity(t-1)', 'rev_activity(t-2)', 'rev_activity(t-3)']

    # Prophet Model
    pr_rev_features = ['c4c_rev', 'gac_rev',
           'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD',
            'gac_activity(t-1)',
           'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
           'gac_activity(t+2)',
           'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
           'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
           'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
           'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
           'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',
                    ]
    
    pr_cm_features = ['c4c_rev', 'gac_rev','rev',
           'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 
            'gac_activity(t-1)',
           'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
           'gac_activity(t+2)', 
           'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
           'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
           'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
           'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
           'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',
           'rev_activity(t-1)', 'rev_activity(t-2)', 'rev_activity(t-3)',]
    

    # Neural Prophet Model

    npr_features = ['c4c_rev', 'gac_rev',
           'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 
            'gac_activity(t-1)',
           'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
           'gac_activity(t+2)', 
           'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
           'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
           'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
           'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
           'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',]


    lagged_features = ['rev','rev_activity(t-1)', 'rev_activity(t-2)', 'rev_activity(t-3)']


    basins = ['AML-DAI',
     'AML-MSP',
     'AML-N/A',
     'AML-NEO',
     'AML-OTH',
     'AML-PRS',
     'AML-RPF',
     'AML-WEC',
     'ASIA-DAI',
     'ASIA-MSP',
     'ASIA-N/A',
     'ASIA-NEO',
     'ASIA-OTH',
     'ASIA-PRS',
     'ASIA-RPF',
     'ASIA-WEC',
     'GHQ-DAI',
     'GHQ-MSP',
     'GHQ-NEO',
     'GHQ-OTH',
     'GHQ-PRS',
     'GHQ-RPF',
     'GHQ-WEC',
     'MENA-DAI',
     'MENA-MSP',
     'MENA-N/A',
     'MENA-NEO',
     'MENA-OTH',
     'MENA-PRS',
     'MENA-RPF',
     'MENA-WEC',
     'N/A-DAI',
     'OAT-DAI',
     'OAT-MSP',
     'OAT-N/A',
     'OAT-NEO',
     'OAT-OTH',
     'OAT-PRS',
     'OAT-RPF',
     'OAT-WEC',
     'RCA-DAI',
     'RCA-MSP',
     'RCA-OTH',
     'RCA-PRS',
     'RCA-RPF',
     'RCA-WEC',
     'Worldwide-DAI',
     'Worldwide-MSP',
     'Worldwide-N/A',
     'Worldwide-NEO',
     'Worldwide-OTH',
     'Worldwide-PRS',
     'Worldwide-RPF',
     'Worldwide-WEC'
        ]
    
    
    # 3. transform the data for Rev
    ## RF
    tt = TrainingTransformer(data_start_date,duration_of_forecast,basins)
    
    
    tt.fit(raw_df)
    
    train_df = tt.transform(raw_df,oil_df)
    
    train_df = train_df[train_df.index<forecast_start_date]
    pr_train_df = train_df.copy()
    pr_train_df = pr_train_df.reset_index()

    it = InferenceRevTransformer(duration_of_forecast,pickup_date,basins)
    it.fit(raw_df)
    infer_df = it.transform(raw_df,oil_df)
    infer_df.to_csv("infer_df.csv")
    pr_infer_df = infer_df.copy()
    pr_infer_df = pr_infer_df.reset_index()

    npr_train_df = pr_train_df.copy()
    npr_infer_df = pr_infer_df.copy()
    npr_infer_df.to_csv("npr_infer_df.csv")

    train_basins = set([
         'MENA-WEC',
         'ASIA-DAI',
#          'OAT-RPF',
#          'AML-WEC',
#          'AML-DAI',
#          'RCA-WEC',
#          'OAT-WEC',
#          'ASIA-RPF',
#          'ASIA-PRS',
#          'ASIA-WEC',
#          'RCA-RPF',
#          'RCA-DAI',
#          'AML-PRS',
#          'AML-RPF',
#          'OAT-PRS',
#          'MENA-PRS',
#          'OAT-DAI',
#          'MENA-RPF',
#          'RCA-PRS',
#          'MENA-DAI',
#         'Worldwide-DAI','Worldwide-WEC','Worldwide-PRS','Worldwide-RPF'
    ])

    # Part-3
    ### RF Model
    rev_model_path = os.path.join(MODEL_PATH, "rev", "RF")
    print('path----',rev_model_path)
    rev_trainer = RfModel(rev_features,rev_var,train_basins,pickup_date,rev_model_path)
    rev_trainer.train(train_df)
    rev_data_dict = rev_trainer.infer(infer_df)


#     pr_rev_model_path = os.path.join(MODEL_PATH, "rev", "Prophet")
#     pr_rev_trainer = ProphetModel(pr_rev_features,pr_rev_features,rev_var,train_basins,pickup_date,pr_rev_model_path)
#     pr_rev_trainer.train(pr_train_df)
#     pr_rev_data_dict = pr_rev_trainer.infer(pr_infer_df)


#     npr_rev_model_path = os.path.join(MODEL_PATH, "rev", "NeuralPr")
#     npr_rev_trainer = NeuralProphetModel(npr_features,npr_features,rev_var[0],[],
#                                          train_basins,pickup_date,rev_model_path)
#     npr_train = npr_rev_trainer.train(npr_train_df)
#     npr_rev_data_dict = npr_rev_trainer.infer(npr_infer_df, npr_train)

    
    ## 4. 
    pickup_date2 = pd.to_datetime(pickup_date)+ relativedelta(months=-12)

    it = InferenceRevTransformer(duration_of_forecast+4,pickup_date2,basins)
    it.fit(raw_df)
    infer_df = it.transform(raw_df,oil_df)

    # 5. Preprocess CM data
    ict = InferenceConMarTrTransformer(rev_data_dict, forecast_start_date)
    ict.fit(infer_df)
    cmf_infer_df = ict.transform(infer_df)    
    cmf_infer_df.to_csv('cmf_infer_df.csv')

    
    # 5.1 

#     icp = InferenceConMarPrTransformer(pr_rev_data_dict,train_basins, forecast_start_date)
#     icp.fit(pr_infer_df)
#     pr_cmf_infer_df = icp.transform(infer_df)

#     icnp = InferenceConMarPrTransformer(npr_rev_data_dict,train_basins,forecast_start_date)
#     icnp.fit(npr_infer_df)
#     npr_cmf_infer_df = icnp.transform(infer_df)
# 5.2
#     npr_cm_model_path = os.path.join(MODEL_PATH, "cm", "NeuralPr")
#     npr_cm_trainer = NeuralProphetModel(npr_features,npr_features,cm_var[0],lagged_features,
#                                          train_basins,pickup_date,npr_cm_model_path)
#     npr_cm_train = npr_cm_trainer.train(npr_train_df)
#     npr_cm_data_dict = npr_cm_trainer.infer(npr_cmf_infer_df,npr_cm_train)
#     npr_cm_data_dict = npr_cm_data_dict.reset_index()[["quarter","SL BASIN (CODE)","cm"]]
#     npr_rev_data_dict = npr_rev_data_dict[["quarter","SL BASIN (CODE)","rev"]]
#     npr_cm_data_dict = npr_cm_data_dict.merge(npr_rev_data_dict[["quarter","SL BASIN (CODE)","rev"]].drop_duplicates(),
#                             on=["quarter","SL BASIN (CODE)"],
#                            how="inner")
    
    
    def convert_dict_to_df(data_dict):
        baseline_data_dict = pd.DataFrame()
        for basin in data_dict.keys():
            df = data_dict[basin]
            df = df.reset_index()
            df["SL BASIN (CODE)"] = basin
            baseline_data_dict = baseline_data_dict.append(df)
        return baseline_data_dict

    ### RF Model
    cm_model_path = os.path.join(MODEL_PATH, "cm","RF")
    cm_trainer = RfModel(cm_features,cm_var,train_basins,pickup_date,cm_model_path)
    cm_trainer.train(train_df)
    cm_data_dict = cm_trainer.infer(cmf_infer_df)
    baseline_data_dict = convert_dict_to_df(cm_data_dict)

#     pr_cm_model_path = os.path.join(MODEL_PATH, "cm","Prophet")
#     pr_cm_trainer = ProphetModel(pr_cm_features,pr_cm_features,cm_var,train_basins,pickup_date,pr_cm_model_path)
#     pr_cm_trainer.train(pr_train_df)
#     pr_cmf_infer_df = pr_cmf_infer_df.reset_index().rename({'quarter':'ds'},axis='columns')
#     pr_cm_data_dict = pr_cm_trainer.infer(pr_cmf_infer_df)
#     pr_cm_data_dict = pr_cm_data_dict.rename({'ds':'quarter'},axis='columns')



    # Ground truth
    ground_truth = train_df[["SL BASIN (CODE)","cm","rev"]].reset_index().drop_duplicates().dropna()
    ground_truth = ground_truth.sort_values(["SL BASIN (CODE)","quarter"])
    ground_truth = ground_truth.rename(mapper={"cm":"actual_contribution_margin","rev":"actual_revenue"},axis=1)
    ground_truth = ground_truth.reset_index().drop(["index"],axis=1)
    ground_truth.to_csv('ground_truth.csv')
    # past dataframes
    try:
        max_date = pd.read_sql("""SELECT max(refresh_date) FROM cmf3_1_mvp3_Q2""",cxn_ODS)
#         max_date = pd.to_datetime(max_date["max(refresh_date)"][0],format="%Y-%m-%d")
        lr_query = """
        SELECT * FROM cmf3_1_mvp3_Q2 WHERE refresh_date >= '{}' 
        """.format(str(max_date)[-6:])
        multi_model_df = pd.read_sql(lr_query,cxn_ODS)
        multi_model_df = multi_model_df[multi_model_df["quarter"]<forecast_start_date]
        multi_model_df = multi_model_df[["quarter","SL BASIN (CODE)","cm","rev","model"]]

    except:
        multi_model_df = pd.DataFrame()
    refresh_date = refresh_date.replace(' ', '') 
    pp = postprocess(refresh_date, cxn_ODS)
    pp.post_process( train_df, baseline_data_dict,"RF", multi_model_df,ground_truth)
#     pp.post_process( train_df, pr_cm_data_dict,"Prophet", multi_model_df,ground_truth)
#     pp.post_process( train_df, npr_cm_data_dict,"Neural Prophet", multi_model_df,ground_truth)

    
    try:
        max_date = pd.read_sql("""SELECT max(refresh_date) FROM cmf3_1_mvp3_Q2""",cxn_ODS)
    #         max_date = pd.to_datetime(max_date["max(refresh_date)"][0],format="%Y-%m-%d")
        lr_query = """
        SELECT * FROM cmf3_1_mvp3_Q2 WHERE refresh_date >= '{}' 
        """.format(str(max_date)[-6:])
        multi_model_df = pd.read_sql(lr_query,cxn_ODS)
        multi_model_df = multi_model_df[multi_model_df["quarter"]<forecast_start_date]
    #     multi_model_df = multi_model_df[["quarter","SL BASIN (CODE)","cm","rev","model","refresh_date"]]

    except:
        multi_model_df = pd.DataFrame()


    model_sel = model_selection(refresh_date,train_basins)
    try:
        best_df = model_sel.best_model(multi_model_df)

        best_df.to_sql(con=cxn_ODS, name='cmf3_1_mvp3_Q2_best',if_exists = 'append',index=None)
    except:
        pass
    
if __name__ == "__main__":
#     counter = 0
#     main(counter)
    for counter in range(4,):
        dff = pd.DataFrame(columns = ['level', 'date', 'quarter', 'actual_revenue', 'predicted_revenue',
       'revenue_error%', 'actual_cm', 'predicted_cm', 'cm_error%',
       'refresh_date'])
        main(counter,dff)
        

---------  Counter:  0


WARNING - (py.warnings._showwarnmsg) - C:\Users\bgondaliya\git\cmf-mvp3-quarter\cmf_engine\fetch_data.py:66: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  c4c_df = pd.read_sql(c4c_query,self.cnx)

WARNING - (py.warnings._showwarnmsg) - C:\Users\bgondaliya\git\cmf-mvp3-quarter\cmf_engine\fetch_data.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gac_df = pd.read_sql(gac_query,self.cnx)

WARNING - (py.warnings._showwarnmsg) - C:\Users\bgondaliya\git\cmf-mvp3-quarter\cmf_engine\fetch_data.py:68: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider 

refresh_date: 2022Q1
path---- C:\Users\bgondaliya\git\cmf-mvp3-quarter\bin\model\rev\RF


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]WARNING - (py.warnings._showwarnmsg) - C:\Users\bgondaliya\git\cmf-mvp3-quarter\cmf_engine\infer.py:75: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X=X_train,y=yt_train)#,sample_weight = weights)



days_before_recent_date: Int64Index([1096, 1096, 1096, 1096, 1096, 1096, 1096, 1096, 1096, 1096,
            ...
               0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
           dtype='int64', length=326)


...

days_before_recent_date: Int64Index([1096, 1096, 1096, 1096, 1096, 1096, 1096, 1096, 1096, 1096,
            ...
               0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
           dtype='int64', length=326)


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.35it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
WARNING - (py.warnings._showwarnmsg) - C:\Users\bgondaliya\AppData\Local\Temp\ipykernel_23848\612050094.py:284: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseline_data_dict = baseline_data_dict.append(df)

WARNING - (py.warnings._showwarnmsg) - C:\Users\bgondaliya\AppData\Local\Temp\ipykernel_23848\612050094.py:284: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future 

Predicting Basin --> ASIA-DAI
Predicting Basin --> MENA-WEC


WARNING - (py.warnings._showwarnmsg) - C:\Users\bgondaliya\git\cmf-mvp3-quarter\cmf_engine\postprocess.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  multi_model_df = multi_model_df.append(data_dict[columns_out])

WARNING - (py.warnings._showwarnmsg) - C:\Users\bgondaliya\git\cmf-mvp3-quarter\cmf_engine\model_selection.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ans = ans.append(pd.merge(df_rev,df_cm, how='left', on = ['quarter','SL BASIN (CODE)']))

WARNING - (py.warnings._showwarnmsg) - C:\Users\bgondaliya\git\cmf-mvp3-quarter\cmf_engine\model_selection.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ans = ans.append(pd.merge(df_rev,df_cm, how='left', on = ['quarter','SL BASIN (CODE)']))



ASIA-DAI
MENA-WEC


In [ ]:
max_date = pd.read_sql("""SELECT max(refresh_date) FROM cmf3_1_mvp3_Q2""",cxn_ODS)
#         max_date = pd.to_datetime(max_date["max(refresh_date)"][0],format="%Y-%m-%d")
lr_query = """
SELECT * FROM cmf3_1_mvp3_Q2  
"""
multi_model_df = pd.read_sql(lr_query,cxn_ODS)
multi_model_df.to_csv('multi_model_df3.csv')
#     multi

In [ ]:
max_date = pd.read_sql("""SELECT max(refresh_date) FROM cmf3_1_mvp3_Q2_best""",cxn_ODS)
#         max_date = pd.to_datetime(max_date["max(refresh_date)"][0],format="%Y-%m-%d")
lr_query = """
SELECT * FROM cmf3_1_mvp3_Q2_best  
"""
multi_model_df = pd.read_sql(lr_query,cxn_ODS)
multi_model_df.to_csv('cmf3_1_mvp3_Q2_best.csv')
#     multi

In [ ]:
max_date = pd.read_sql("""SELECT max(refresh_date) FROM cmf3_1_mvp3_Q1_best""",cxn_ODS)
#         max_date = pd.to_datetime(max_date["max(refresh_date)"][0],format="%Y-%m-%d")
lr_query = """
SELECT * FROM cmf3_1_mvp3_Q1_best  
"""
multi_model_df = pd.read_sql(lr_query,cxn_ODS)
multi_model_df.to_csv('cmf3_1_mvp3_Q1_best.csv')
#     multi

# Model Selection

In [ ]:
counter=4
print("---------  Counter: ", counter)
path = 'C:/Users/bgondaliya/git/cmf-mvp3-quarter/bin/local/'

# 0. get pond Credentials
cred = credentials(path)
cxn_ODS,cxn_6438 = cred.read_cred()

# 1. to get the data
data = TeleportData(cxn_6438,cxn_ODS)
raw_df = data.get_data()
oil_df = data.get_eia_data(cxn_ODS)
#     raw_df.to_csv('raw_df.csv')
#     oil_df.to_csv('oil_df.csv')

# 2.define variables
basins = ['AML', 'ASIA', 'MENA', 'OAT', 'RCA', 'Worldwide']




data_start_date = "2017-01-01"
pickup_date = pd.to_datetime("2022-02-01",format="%Y-%m-%d")+ relativedelta(months=counter*3)#str(datetime.date(date.today().year, date.today().month, 1))
quarter = (pickup_date.month - 1) // 3 + 1
forecast_start_date = pickup_date.strftime('%YQ{}'.format(quarter))
refresh_date =  pd.to_datetime("2022-02-16",format="%Y-%m-%d")+ relativedelta(months=counter*3) #str(date.today())
refresh_date = refresh_date.strftime('%YQ{}'.format(quarter))
duration_of_forecast = 4 #quarter


print("refresh_date:",refresh_date)
#     data_start_date = "2020-05-01"
#     forecast_start_date = str(pd.to_datetime("2022-03-01",format="%Y-%m-%d")+ relativedelta(months=counter))#str(datetime.date(date.today().year, date.today().month, 1))
#     refresh_date =  str(pd.to_datetime("2022-03-26",format="%Y-%m-%d")+ relativedelta(months=counter)) #str(date.today())
#     duration_of_forecast = 12


raw_df = raw_df[raw_df["SL BASIN (CODE)"]!="SL_Elims"]
raw_df["SL BASIN (CODE)"] = raw_df["SL BASIN (CODE)"]+"-"+raw_df["SL DIVISION (CODE)"]


# Dep Vars
rev_var = ["rev"]
cm_var = ['cm']

# Baseline Model

rev_features = ['c4c_rev', 'gac_rev',
       'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 'gac_activity(t-1)',
       'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
       'gac_activity(t+2)',
       'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
       'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
       'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
       'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
       'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)']



cm_features = [
        'rev', 'c4c_rev', 'gac_rev',
       'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 'gac_activity(t-1)',
       'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
       'gac_activity(t+2)',
       'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
       'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
       'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
       'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
       'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',
       'rev_activity(t-1)', 'rev_activity(t-2)', 'rev_activity(t-3)']

# Prophet Model
pr_rev_features = ['c4c_rev', 'gac_rev',
       'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD',
        'gac_activity(t-1)',
       'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
       'gac_activity(t+2)',
       'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
       'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
       'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
       'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
       'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',
                ]

pr_cm_features = ['c4c_rev', 'gac_rev','rev',
       'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 
        'gac_activity(t-1)',
       'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
       'gac_activity(t+2)', 
       'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
       'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
       'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
       'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
       'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',
       'rev_activity(t-1)', 'rev_activity(t-2)', 'rev_activity(t-3)',]


# Neural Prophet Model

npr_features = ['c4c_rev', 'gac_rev',
       'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 
        'gac_activity(t-1)',
       'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
       'gac_activity(t+2)', 
       'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
       'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
       'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
       'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
       'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',]


lagged_features = ['rev','rev_activity(t-1)', 'rev_activity(t-2)', 'rev_activity(t-3)']


basins = ['AML-DAI',
 'AML-MSP',
 'AML-N/A',
 'AML-NEO',
 'AML-OTH',
 'AML-PRS',
 'AML-RPF',
 'AML-WEC',
 'ASIA-DAI',
 'ASIA-MSP',
 'ASIA-N/A',
 'ASIA-NEO',
 'ASIA-OTH',
 'ASIA-PRS',
 'ASIA-RPF',
 'ASIA-WEC',
 'GHQ-DAI',
 'GHQ-MSP',
 'GHQ-NEO',
 'GHQ-OTH',
 'GHQ-PRS',
 'GHQ-RPF',
 'GHQ-WEC',
 'MENA-DAI',
 'MENA-MSP',
 'MENA-N/A',
 'MENA-NEO',
 'MENA-OTH',
 'MENA-PRS',
 'MENA-RPF',
 'MENA-WEC',
 'N/A-DAI',
 'OAT-DAI',
 'OAT-MSP',
 'OAT-N/A',
 'OAT-NEO',
 'OAT-OTH',
 'OAT-PRS',
 'OAT-RPF',
 'OAT-WEC',
 'RCA-DAI',
 'RCA-MSP',
 'RCA-OTH',
 'RCA-PRS',
 'RCA-RPF',
 'RCA-WEC',
 'Worldwide-DAI',
 'Worldwide-MSP',
 'Worldwide-N/A',
 'Worldwide-NEO',
 'Worldwide-OTH',
 'Worldwide-PRS',
 'Worldwide-RPF',
 'Worldwide-WEC'
    ]


# 3. transform the data for Rev
## RF
tt = TrainingTransformer(data_start_date,duration_of_forecast,basins)


tt.fit(raw_df)

train_df = tt.transform(raw_df,oil_df)

train_df = train_df[train_df.index<forecast_start_date]
pr_train_df = train_df.copy()
pr_train_df = pr_train_df.reset_index()

it = InferenceRevTransformer(duration_of_forecast,pickup_date,basins)
it.fit(raw_df)
infer_df = it.transform(raw_df,oil_df)
infer_df.to_csv("infer_df.csv")
pr_infer_df = infer_df.copy()
pr_infer_df = pr_infer_df.reset_index()

npr_train_df = pr_train_df.copy()
npr_infer_df = pr_infer_df.copy()
npr_infer_df.to_csv("npr_infer_df.csv")

train_basins = set([
     'MENA-WEC',
     'ASIA-DAI',
#          'OAT-RPF',
#          'AML-WEC',
#          'AML-DAI',
#          'RCA-WEC',
#          'OAT-WEC',
#          'ASIA-RPF',
#          'ASIA-PRS',
#          'ASIA-WEC',
#          'RCA-RPF',
#          'RCA-DAI',
#          'AML-PRS',
#          'AML-RPF',
#          'OAT-PRS',
#          'MENA-PRS',
#          'OAT-DAI',
#          'MENA-RPF',
#          'RCA-PRS',
#          'MENA-DAI',
#         'Worldwide-DAI','Worldwide-WEC','Worldwide-PRS','Worldwide-RPF'
])

# Part-3
### RF Model
rev_model_path = os.path.join(MODEL_PATH, "rev", "RF")
print('path----',rev_model_path)
rev_trainer = RfModel(rev_features,rev_var,train_basins,pickup_date,rev_model_path)
rev_trainer.train(train_df)
rev_data_dict = rev_trainer.infer(infer_df)


#     pr_rev_model_path = os.path.join(MODEL_PATH, "rev", "Prophet")
#     pr_rev_trainer = ProphetModel(pr_rev_features,pr_rev_features,rev_var,train_basins,pickup_date,pr_rev_model_path)
#     pr_rev_trainer.train(pr_train_df)
#     pr_rev_data_dict = pr_rev_trainer.infer(pr_infer_df)


#     npr_rev_model_path = os.path.join(MODEL_PATH, "rev", "NeuralPr")
#     npr_rev_trainer = NeuralProphetModel(npr_features,npr_features,rev_var[0],[],
#                                          train_basins,pickup_date,rev_model_path)
#     npr_train = npr_rev_trainer.train(npr_train_df)
#     npr_rev_data_dict = npr_rev_trainer.infer(npr_infer_df, npr_train)


## 4. 
pickup_date2 = pd.to_datetime(pickup_date)+ relativedelta(months=-12)

it = InferenceRevTransformer(duration_of_forecast+4,pickup_date2,basins)
it.fit(raw_df)
infer_df = it.transform(raw_df,oil_df)

# 5. Preprocess CM data
ict = InferenceConMarTrTransformer(rev_data_dict, forecast_start_date)
ict.fit(infer_df)
cmf_infer_df = ict.transform(infer_df)    
cmf_infer_df.to_csv('cmf_infer_df.csv')


# 5.1 

#     icp = InferenceConMarPrTransformer(pr_rev_data_dict,train_basins, forecast_start_date)
#     icp.fit(pr_infer_df)
#     pr_cmf_infer_df = icp.transform(infer_df)

#     icnp = InferenceConMarPrTransformer(npr_rev_data_dict,train_basins,forecast_start_date)
#     icnp.fit(npr_infer_df)
#     npr_cmf_infer_df = icnp.transform(infer_df)
# 5.2
#     npr_cm_model_path = os.path.join(MODEL_PATH, "cm", "NeuralPr")
#     npr_cm_trainer = NeuralProphetModel(npr_features,npr_features,cm_var[0],lagged_features,
#                                          train_basins,pickup_date,npr_cm_model_path)
#     npr_cm_train = npr_cm_trainer.train(npr_train_df)
#     npr_cm_data_dict = npr_cm_trainer.infer(npr_cmf_infer_df,npr_cm_train)
#     npr_cm_data_dict = npr_cm_data_dict.reset_index()[["quarter","SL BASIN (CODE)","cm"]]
#     npr_rev_data_dict = npr_rev_data_dict[["quarter","SL BASIN (CODE)","rev"]]
#     npr_cm_data_dict = npr_cm_data_dict.merge(npr_rev_data_dict[["quarter","SL BASIN (CODE)","rev"]].drop_duplicates(),
#                             on=["quarter","SL BASIN (CODE)"],
#                            how="inner")


def convert_dict_to_df(data_dict):
    baseline_data_dict = pd.DataFrame()
    for basin in data_dict.keys():
        df = data_dict[basin]
        df = df.reset_index()
        df["SL BASIN (CODE)"] = basin
        baseline_data_dict = baseline_data_dict.append(df)
    return baseline_data_dict

### RF Model
cm_model_path = os.path.join(MODEL_PATH, "cm","RF")
cm_trainer = RfModel(cm_features,cm_var,train_basins,pickup_date,cm_model_path)
cm_trainer.train(train_df)
cm_data_dict = cm_trainer.infer(cmf_infer_df)
baseline_data_dict = convert_dict_to_df(cm_data_dict)

#     pr_cm_model_path = os.path.join(MODEL_PATH, "cm","Prophet")
#     pr_cm_trainer = ProphetModel(pr_cm_features,pr_cm_features,cm_var,train_basins,pickup_date,pr_cm_model_path)
#     pr_cm_trainer.train(pr_train_df)
#     pr_cmf_infer_df = pr_cmf_infer_df.reset_index().rename({'quarter':'ds'},axis='columns')
#     pr_cm_data_dict = pr_cm_trainer.infer(pr_cmf_infer_df)
#     pr_cm_data_dict = pr_cm_data_dict.rename({'ds':'quarter'},axis='columns')



# Ground truth
ground_truth = train_df[["SL BASIN (CODE)","cm","rev"]].reset_index().drop_duplicates().dropna()
ground_truth = ground_truth.sort_values(["SL BASIN (CODE)","quarter"])
ground_truth = ground_truth.rename(mapper={"cm":"actual_contribution_margin","rev":"actual_revenue"},axis=1)
ground_truth = ground_truth.reset_index().drop(["index"],axis=1)
ground_truth.to_csv('ground_truth.csv')
# past dataframes
try:
    max_date = pd.read_sql("""SELECT max(refresh_date) FROM cmf3_1_mvp3_Q2""",cxn_ODS)
#         max_date = pd.to_datetime(max_date["max(refresh_date)"][0],format="%Y-%m-%d")
    lr_query = """
    SELECT * FROM cmf3_1_mvp3_Q2 WHERE refresh_date >= '{}' 
    """.format(str(max_date)[-6:])
    multi_model_df = pd.read_sql(lr_query,cxn_ODS)
    multi_model_df = multi_model_df[multi_model_df["quarter"]<forecast_start_date]
    multi_model_df = multi_model_df[["quarter","SL BASIN (CODE)","cm","rev","model"]]

except:
    multi_model_df = pd.DataFrame()
refresh_date = refresh_date.replace(' ', '') 
pp = postprocess(refresh_date, cxn_ODS)
pp.post_process( train_df, baseline_data_dict,"RF", multi_model_df,ground_truth)
#     pp.post_process( train_df, pr_cm_data_dict,"Prophet", multi_model_df,ground_truth)
#     pp.post_process( train_df, npr_cm_data_dict,"Neural Prophet", multi_model_df,ground_truth)


try:
    max_date = pd.read_sql("""SELECT max(refresh_date) FROM cmf3_1_mvp3_Q2""",cxn_ODS)
#         max_date = pd.to_datetime(max_date["max(refresh_date)"][0],format="%Y-%m-%d")
    lr_query = """
    SELECT * FROM cmf3_1_mvp3_Q2 WHERE refresh_date >= '{}' 
    """.format(str(max_date)[-6:])
    multi_model_df = pd.read_sql(lr_query,cxn_ODS)
    multi_model_df = multi_model_df[multi_model_df["quarter"]<forecast_start_date]
#     multi_model_df = multi_model_df[["quarter","SL BASIN (CODE)","cm","rev","model","refresh_date"]]

except:
    multi_model_df = pd.DataFrame()


In [ ]:
multi_model_df

In [ ]:
model_sel = model_selection(refresh_date,train_basins)
model_sel.best_model(multi_model_df)

In [ ]:


model_sel = model_selection(refresh_date,train_basins)
try:
    best_df = model_sel.best_model(multi_model_df)

    best_df.to_sql(con=cxn_ODS, name='cmf3_1_mvp3_Q2_best',if_exists = 'append',index=None)
except:
    pass


In [ ]:
best_df

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  6 11:17:50 2023

@author: BGondaliya
"""

import datetime
from datetime import date
import pandas as pd
from dateutil.relativedelta import relativedelta


class model_selection:
    def __init__(self,refresh_date,train_basins):
        self.ref_date = refresh_date
        self.refresh_date = str(pd.to_datetime(refresh_date).replace(day=1)) #str(date.today(
        temp = pd.to_datetime(refresh_date) + relativedelta(months=-12)
        self.cutoff_date = temp.strftime('%YQ{}'.format( (temp.month - 1) // 3 + 1))
        self.train_basins = train_basins


    def best_model(self, X):
        data = X[X['refresh_date']== self.ref_date]
        data = data[(data['quarter'] > self.cutoff_date) & (data['quarter'] < self.ref_date)]

        data['abs_rev_error'] = abs(data['revenue_error'])
        data['abs_cm_error'] = abs(data['contribution_margin_error'])
        
        grouped_df = data.groupby(['SL BASIN (CODE)', 'model','refresh_date']).agg({'abs_rev_error': 'mean', 'abs_cm_error': 'mean'})
        grouped_df = grouped_df.dropna()
        
        grouped = grouped_df.groupby('SL BASIN (CODE)')
        min_indices = grouped['abs_rev_error'].idxmin()
        best_rev_df = grouped_df.loc[min_indices]
        
        grouped = grouped_df.groupby('SL BASIN (CODE)')

        min_indices = grouped['abs_cm_error'].idxmin()
        best_cm_df = grouped_df.loc[min_indices]
        
        # to extract dataframe for best model
        data = X[X['refresh_date']== self.ref_date]
#         data = data[data['refresh_date']==self.refresh_date]
        ans = pd.DataFrame(columns=['quarter', 'SL BASIN (CODE)', 'rev', 'actual_revenue',
               'revenue_error', 'YM', 'refresh_date', 'level', 'rev_Model', 'cm',
               'actual_contribution_margin', 'contribution_margin_error', 'cm_Model'])   
        for basin in self.train_basins:
            print(basin)

            rev_model = best_rev_df.loc[best_rev_df.index.get_level_values('SL BASIN (CODE)') == basin].index.get_level_values('model')
            cm_model = best_cm_df.loc[best_cm_df.index.get_level_values('SL BASIN (CODE)') == basin].index.get_level_values('model')

            df_rev = data[(data['SL BASIN (CODE)']==basin) &(data['model']=="RF")][['quarter', 'SL BASIN (CODE)', 'rev',
                    'actual_revenue','revenue_error', 'YM', 'refresh_date',
                   'level']]
            df_rev['rev_Model'] = str(rev_model[0])
            df_rev = df_rev.drop_duplicates()

            df_cm = data[(data['SL BASIN (CODE)']==basin) &(data['model']=="RF")][['quarter', 'SL BASIN (CODE)', 'cm',
                   'actual_contribution_margin', 
                   'contribution_margin_error', ]]
            df_cm['cm_Model'] = str(cm_model[0])
            df_cm = df_cm.drop_duplicates()

            ans = ans.append(pd.merge(df_rev,df_cm, how='left', on = ['quarter','SL BASIN (CODE)']))

        return ans

In [ ]:


model_sel = model_selection(refresh_date,train_basins)
try:
    best_df = model_sel.best_model(multi_model_df)

    best_df.to_sql(con=cxn_ODS, name='cmf3_1_mvp3_Q2_best',if_exists = 'append',index=None)
except:
    pass

In [ ]:
data_start_date = "2017-01-01"
pickup_date = pd.to_datetime("2022-02-01",format="%Y-%m-%d")+ relativedelta(months=counter*3)#str(datetime.date(date.today().year, date.today().month, 1))
quarter = (pickup_date.month - 1) // 3 + 1
forecast_start_date = pickup_date.strftime('%YQ{}'.format(quarter))
refresh_date =  pd.to_datetime("2022-02-16",format="%Y-%m-%d")+ relativedelta(months=counter*3) #str(date.today())
refresh_date = refresh_date.strftime('%YQ{}'.format(quarter))
duration_of_forecast = 4 #quarter


In [ ]:
max_date = pd.read_sql("""SELECT max(refresh_date) FROM cmf3_1_mvp3_Q2""",cxn_ODS)
#         max_date = pd.to_datetime(max_date["max(refresh_date)"][0],format="%Y-%m-%d")
lr_query = """
SELECT * FROM cmf3_1_mvp3_Q2 WHERE refresh_date >= '{}' 
""".format(str(max_date)[-6:])
multi_model_df = pd.read_sql(lr_query,cxn_ODS)
multi_model_df = multi_model_df[multi_model_df["quarter"]<forecast_start_date]
#     multi_model_df = multi_model_df[["quarter","SL BASIN (CODE)","cm","rev","model","refresh_date"]]

In [ ]:
multi_model_df

In [ ]:
model_sel = model_selection(refresh_date,train_basins)

In [ ]:
multi_model_df

In [ ]:
ref_date = refresh_date
refresh_date = str(pd.to_datetime(refresh_date).replace(day=1)) 
cutoff_date = pd.to_datetime(refresh_date) + relativedelta(months=-12)
cutoff_date = cutoff_date.strftime('%YQ{}'.format( (cutoff_date.month - 1) // 3 + 1))
train_basins = train_basins

In [ ]:
cutoff_date

In [ ]:
ref_date

In [ ]:
refresh_date

In [ ]:
X = multi_model_df.copy()

In [ ]:
X

In [ ]:
X[X['refresh_date']== ref_date]

In [ ]:
data = X[X['refresh_date']== ref_date]
data = data[(data['quarter'] > cutoff_date) & (data['quarter'] < ref_date)]

data['abs_rev_error'] = abs(data['revenue_error'])
data['abs_cm_error'] = abs(data['contribution_margin_error'])

In [ ]:
data

In [ ]:
grouped_df = data.groupby(['SL BASIN (CODE)', 'model','refresh_date']).agg({'abs_rev_error': 'mean', 'abs_cm_error': 'mean'})
grouped_df = grouped_df.dropna()
grouped_df 

In [ ]:
grouped = grouped_df.groupby('SL BASIN (CODE)')
min_indices = grouped['abs_rev_error'].idxmin()
min_indices

In [ ]:

best_rev_df = grouped_df.loc[min_indices]



In [ ]:
best_rev_df

In [ ]:

grouped = grouped_df.groupby('SL BASIN (CODE)')

min_indices = grouped['abs_cm_error'].idxmin()
best_cm_df = grouped_df.loc[min_indices]

# to extract dataframe for best model
data = X[X['refresh_date']== ref_date]
#         data = data[data['refresh_date']==self.refresh_date]
ans = pd.DataFrame(columns=['YEARMONTH', 'SL BASIN (CODE)', 'rev', 'actual_revenue',
       'revenue_error', 'YM', 'refresh_date', 'level', 'rev_Model', 'cm',
       'actual_contribution_margin', 'contribution_margin_error', 'cm_Model'])   


In [ ]:
best_cm_df

In [ ]:
train_basins

In [ ]:
basin ='ASIA-DAI'

In [ ]:
rev_model = best_rev_df.loc[best_rev_df.index.get_level_values('SL BASIN (CODE)') == basin].index.get_level_values('model')
      

In [ ]:
cm_model = best_cm_df.loc[best_cm_df.index.get_level_values('SL BASIN (CODE)') == basin].index.get_level_values('model')


In [ ]:
# data[data['model']==best_cm_df[0]]

In [ ]:
# best_cm_df = best_cm_df.reset_index()

In [ ]:
df_rev = data[(data['SL BASIN (CODE)']==basin) & (data['model']==rev_model[0])][['quarter', 'SL BASIN (CODE)', 'rev',
                    'actual_revenue','revenue_error', 'YM', 'refresh_date',
                   'level']]

In [ ]:
df_rev

In [ ]:
best_cm_df.head()

In [ ]:
best_cm_df[(best_cm_df['SL BASIN (CODE)']==basin) &(best_cm_df['model']=="RF")]

In [ ]:
best_cm_df[(best_cm_df['SL BASIN (CODE)']==basin) &(best_cm_df['model']=="RF")][['SL BASIN (CODE)',
                'abs_rev_error','abs_cm_error',  'refresh_date']]


In [ ]:
best_cm_df

In [ ]:
best_cm_df.loc[best_cm_df.index.get_level_values('SL BASIN (CODE)') == basin].index.get_level_values('model')
        

In [ ]:
best_rev_df

In [ ]:
best_rev_df[(best_rev_df.index.get_level_values('SL BASIN (CODE)')==basin) &(best_rev_df.index.get_level_values('model')==rev_model[0])][[
                'abs_rev_error','abs_cm_error']]

In [ ]:
for basin in train_basins:
        print(basin)

        rev_model = best_rev_df.loc[best_rev_df.index.get_level_values('SL BASIN (CODE)') == basin].index.get_level_values('model')
        cm_model = best_cm_df.loc[best_cm_df.index.get_level_values('SL BASIN (CODE)') == basin].index.get_level_values('model')
        
#         best_cm_df = best_cm_df.reset_index()
#         best_rev_df = best_rev_df.reset_index()
        
        df_rev = best_rev_df[(best_rev_df.index.get_level_values('SL BASIN (CODE)')==basin) &(best_rev_df.index.get_level_values('model')==rev_model[0])][[
                'abs_rev_error','abs_cm_error']]

        df_rev['rev_Model'] = str(rev_model[0])
        df_rev = df_rev.drop_duplicates()

        df_cm = best_cm_df[(best_cm_df.index.get_level_values('SL BASIN (CODE)')==basin) &(best_cm_df.index.get_level_values('model')==cm_model[0])][[
                'abs_rev_error','abs_cm_error']]
        df_cm['cm_Model'] = str(cm_model[0])
        df_cm = df_cm.drop_duplicates()
        break
#         ans = ans.append(pd.merge(df_rev,df_cm, how='left', on = ['quarter','SL BASIN (CODE)']))

In [ ]:
df_cm.reset_index()

In [ ]:
pd.merge(df_rev.reset_index(),df_cm.reset_index()[, how='left', on = ['SL BASIN (CODE)'])

In [ ]:
ans = pd.DataFrame(columns=[ 'SL BASIN (CODE)', 'rev', 'actual_revenue',
               'revenue_error', 'YM', 'refresh_date', 'level', 'rev_Model', 'cm',
               'actual_contribution_margin', 'contribution_margin_error', 'cm_Model'])   
ans

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  6 11:17:50 2023

@author: BGondaliya
"""

import datetime
from datetime import date
import pandas as pd
from dateutil.relativedelta import relativedelta


class model_selection:
    def __init__(self,refresh_date,train_basins):
        self.ref_date = refresh_date
        self.refresh_date = str(pd.to_datetime(refresh_date).replace(day=1)) #str(date.today(
        temp = pd.to_datetime(refresh_date) + relativedelta(months=-12)
        self.cutoff_date = temp.strftime('%YQ{}'.format( (temp.month - 1) // 3 + 1))
        self.train_basins = train_basins


    def best_model(self, X):
        data = X[X['refresh_date']== self.ref_date]
        data = data[(data['quarter'] > self.cutoff_date) & (data['quarter'] < self.ref_date)]

        data['abs_rev_error'] = abs(data['revenue_error'])
        data['abs_cm_error'] = abs(data['contribution_margin_error'])
        
        grouped_df = data.groupby(['SL BASIN (CODE)', 'model','refresh_date']).agg({'abs_rev_error': 'mean', 'abs_cm_error': 'mean'})
        grouped_df = grouped_df.dropna()
        
        grouped = grouped_df.groupby('SL BASIN (CODE)')
        min_indices = grouped['abs_rev_error'].idxmin()
        best_rev_df = grouped_df.loc[min_indices]
        
        grouped = grouped_df.groupby('SL BASIN (CODE)')

        min_indices = grouped['abs_cm_error'].idxmin()
        best_cm_df = grouped_df.loc[min_indices]
        
        # to extract dataframe for best model
        data = X[X['refresh_date']== self.ref_date]
#         data = data[data['refresh_date']==self.refresh_date]
        ans = pd.DataFrame(columns=['quarter', 'SL BASIN (CODE)', 'rev', 'actual_revenue',
               'revenue_error', 'YM', 'refresh_date', 'level', 'rev_Model', 'cm',
               'actual_contribution_margin', 'contribution_margin_error', 'cm_Model'])   
        for basin in self.train_basins:
            print(basin)

            rev_model = best_rev_df.loc[best_rev_df.index.get_level_values('SL BASIN (CODE)') == basin].index.get_level_values('model')
            cm_model = best_cm_df.loc[best_cm_df.index.get_level_values('SL BASIN (CODE)') == basin].index.get_level_values('model')

            df_rev = data[(data['SL BASIN (CODE)']==basin) &(data['model']==rev_model[0])][['quarter', 'SL BASIN (CODE)', 'rev',
                    'actual_revenue','revenue_error', 'YM', 'refresh_date',
                   'level']]
            df_rev['rev_Model'] = str(rev_model[0])
            df_rev = df_rev.drop_duplicates()

            df_cm = data[(data['SL BASIN (CODE)']==basin) &(data['model']==cm_model[0])][['quarter', 'SL BASIN (CODE)', 'cm',
                   'actual_contribution_margin', 
                   'contribution_margin_error', ]]
            df_cm['cm_Model'] = str(cm_model[0])
            df_cm = df_cm.drop_duplicates()

            ans = ans.append(pd.merge(df_rev,df_cm, how='left', on = ['quarter','SL BASIN (CODE)']))

        return ans

In [ ]:
model_sel.best_model(multi_model_df)

In [ ]:


try:
    best_df = model_sel.best_model(multi_model_df)

    best_df.to_sql(con=cxn_ODS, name='cmf3_1_mvp3_Q2_best',if_exists = 'append',index=None)
except:
    pass


In [ ]:
best_df.to_csv('best_df.csv')

In [ ]:
multi_model_df.to_csv('multi_model_df2.csv')

In [ ]:
print("---------  Counter: ", counter)
path = 'C:/Users/bgondaliya/git/cmf-mvp3-quarter/bin/local/'

# 0. get pond Credentials
cred = credentials(path)
cxn_ODS,cxn_6438 = cred.read_cred()


In [ ]:
max_date = pd.read_sql("""SELECT max(refresh_date) FROM cmf3_1_mvp3_Q2""",cxn_ODS)
#         max_date = pd.to_datetime(max_date["max(refresh_date)"][0],format="%Y-%m-%d")
lr_query = """
SELECT * FROM cmf3_1_mvp3_Q2 
"""
multi_model_df = pd.read_sql(lr_query,cxn_ODS)


In [ ]:
multi_model_df.to_csv('multi_model_df.csv')

In [ ]:
multi_model_df

In [ ]:
model_sel.best_model(multi_model_df)

In [ ]:
model_sel = model_selection(refresh_date,train_basins)
try:
    best_df = model_sel.best_model(multi_model_df)

    best_df.to_sql(con=cxn_ODS, name='cmf3_1_mvp3_Q2_best',if_exists = 'append',index=None)
except:
    pass


In [ ]:
counter=5
print("---------  Counter: ", counter)
path = 'C:/Users/bgondaliya/git/cmf-mvp3-quarter/bin/local/'

# 0. get pond Credentials
cred = credentials(path)
cxn_ODS,cxn_6438 = cred.read_cred()

# 1. to get the data
data = TeleportData(cxn_6438,cxn_ODS)
raw_df = data.get_data()
oil_df = data.get_eia_data(cxn_ODS)
#     raw_df.to_csv('raw_df.csv')
#     oil_df.to_csv('oil_df.csv')

# 2.define variables
basins = ['AML', 'ASIA', 'MENA', 'OAT', 'RCA', 'Worldwide']




data_start_date = "2017-01-01"
pickup_date = pd.to_datetime("2022-02-01",format="%Y-%m-%d")+ relativedelta(months=counter*3)#str(datetime.date(date.today().year, date.today().month, 1))
quarter = (pickup_date.month - 1) // 3 + 1
forecast_start_date = pickup_date.strftime('%YQ{}'.format(quarter))
refresh_date =  pd.to_datetime("2022-02-16",format="%Y-%m-%d")+ relativedelta(months=counter*3) #str(date.today())
refresh_date = refresh_date.strftime('%YQ{}'.format(quarter))
duration_of_forecast = 4 #quarter


print("refresh_date:",refresh_date)
#     data_start_date = "2020-05-01"
#     forecast_start_date = str(pd.to_datetime("2022-03-01",format="%Y-%m-%d")+ relativedelta(months=counter))#str(datetime.date(date.today().year, date.today().month, 1))
#     refresh_date =  str(pd.to_datetime("2022-03-26",format="%Y-%m-%d")+ relativedelta(months=counter)) #str(date.today())
#     duration_of_forecast = 12


raw_df = raw_df[raw_df["SL BASIN (CODE)"]!="SL_Elims"]
raw_df["SL BASIN (CODE)"] = raw_df["SL BASIN (CODE)"]+"-"+raw_df["SL DIVISION (CODE)"]


# Dep Vars
rev_var = ["rev"]
cm_var = ['cm']

# Baseline Model

rev_features = ['c4c_rev', 'gac_rev',
       'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 'gac_activity(t-1)',
       'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
       'gac_activity(t+2)',
       'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
       'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
       'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
       'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
       'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)']



cm_features = [
        'rev', 'c4c_rev', 'gac_rev',
       'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 'gac_activity(t-1)',
       'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
       'gac_activity(t+2)',
       'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
       'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
       'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
       'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
       'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',
       'rev_activity(t-1)', 'rev_activity(t-2)', 'rev_activity(t-3)']

# Prophet Model
pr_rev_features = ['c4c_rev', 'gac_rev',
       'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD',
        'gac_activity(t-1)',
       'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
       'gac_activity(t+2)',
       'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
       'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
       'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
       'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
       'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',
                ]

pr_cm_features = ['c4c_rev', 'gac_rev','rev',
       'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 
        'gac_activity(t-1)',
       'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
       'gac_activity(t+2)', 
       'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
       'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
       'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
       'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
       'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',
       'rev_activity(t-1)', 'rev_activity(t-2)', 'rev_activity(t-3)',]


# Neural Prophet Model

npr_features = ['c4c_rev', 'gac_rev',
       'BC_OIL_PRICE_USD', 'WTI_OIL_PRICE_USD', 
        'gac_activity(t-1)',
       'gac_activity(t-2)', 'gac_activity(t-3)', 'gac_activity(t+1)',
       'gac_activity(t+2)', 
       'c4c_activity(t-1)', 'c4c_activity(t-2)', 'c4c_activity(t-3)',
       'c4c_activity(t+1)', 'c4c_activity(t+2)', 'wti_activity(t-1)',
       'wti_activity(t-2)', 'wti_activity(t-3)', 'wti_activity(t+1)',
       'wti_activity(t+2)', 'btc_activity(t-1)', 'btc_activity(t-2)',
       'btc_activity(t-3)', 'btc_activity(t+1)', 'btc_activity(t+2)',]


lagged_features = ['rev','rev_activity(t-1)', 'rev_activity(t-2)', 'rev_activity(t-3)']


basins = ['AML-DAI',
 'AML-MSP',
 'AML-N/A',
 'AML-NEO',
 'AML-OTH',
 'AML-PRS',
 'AML-RPF',
 'AML-WEC',
 'ASIA-DAI',
 'ASIA-MSP',
 'ASIA-N/A',
 'ASIA-NEO',
 'ASIA-OTH',
 'ASIA-PRS',
 'ASIA-RPF',
 'ASIA-WEC',
 'GHQ-DAI',
 'GHQ-MSP',
 'GHQ-NEO',
 'GHQ-OTH',
 'GHQ-PRS',
 'GHQ-RPF',
 'GHQ-WEC',
 'MENA-DAI',
 'MENA-MSP',
 'MENA-N/A',
 'MENA-NEO',
 'MENA-OTH',
 'MENA-PRS',
 'MENA-RPF',
 'MENA-WEC',
 'N/A-DAI',
 'OAT-DAI',
 'OAT-MSP',
 'OAT-N/A',
 'OAT-NEO',
 'OAT-OTH',
 'OAT-PRS',
 'OAT-RPF',
 'OAT-WEC',
 'RCA-DAI',
 'RCA-MSP',
 'RCA-OTH',
 'RCA-PRS',
 'RCA-RPF',
 'RCA-WEC',
 'Worldwide-DAI',
 'Worldwide-MSP',
 'Worldwide-N/A',
 'Worldwide-NEO',
 'Worldwide-OTH',
 'Worldwide-PRS',
 'Worldwide-RPF',
 'Worldwide-WEC'
    ]


# 3. transform the data for Rev
## RF
tt = TrainingTransformer(data_start_date,duration_of_forecast,basins)


tt.fit(raw_df)

train_df = tt.transform(raw_df,oil_df)

train_df = train_df[train_df.index<forecast_start_date]
pr_train_df = train_df.copy()
pr_train_df = pr_train_df.reset_index()

it = InferenceRevTransformer(duration_of_forecast,pickup_date,basins)
it.fit(raw_df)
infer_df = it.transform(raw_df,oil_df)
infer_df.to_csv("infer_df.csv")
pr_infer_df = infer_df.copy()
pr_infer_df = pr_infer_df.reset_index()

npr_train_df = pr_train_df.copy()
npr_infer_df = pr_infer_df.copy()
npr_infer_df.to_csv("npr_infer_df.csv")

train_basins = set([
     'MENA-WEC',
     'ASIA-DAI',
     'OAT-RPF',
     'AML-WEC',
     'AML-DAI',
     'RCA-WEC',
     'OAT-WEC',
     'ASIA-RPF',
     'ASIA-PRS',
     'ASIA-WEC',
     'RCA-RPF',
     'RCA-DAI',
     'AML-PRS',
     'AML-RPF',
     'OAT-PRS',
     'MENA-PRS',
     'OAT-DAI',
     'MENA-RPF',
     'RCA-PRS',
     'MENA-DAI',
    'Worldwide-DAI','Worldwide-WEC','Worldwide-PRS','Worldwide-RPF'
])

# Part-3
### RF Model
rev_model_path = os.path.join(MODEL_PATH, "rev", "RF")
print('path----',rev_model_path)
rev_trainer = RfModel(rev_features,rev_var,train_basins,pickup_date,rev_model_path)
rev_trainer.train(train_df)
rev_data_dict = rev_trainer.infer(infer_df)


#     pr_rev_model_path = os.path.join(MODEL_PATH, "rev", "Prophet")
#     pr_rev_trainer = ProphetModel(pr_rev_features,pr_rev_features,rev_var,train_basins,pickup_date,pr_rev_model_path)
#     pr_rev_trainer.train(pr_train_df)
#     pr_rev_data_dict = pr_rev_trainer.infer(pr_infer_df)


#     npr_rev_model_path = os.path.join(MODEL_PATH, "rev", "NeuralPr")
#     npr_rev_trainer = NeuralProphetModel(npr_features,npr_features,rev_var[0],[],
#                                          train_basins,pickup_date,rev_model_path)
#     npr_train = npr_rev_trainer.train(npr_train_df)
#     npr_rev_data_dict = npr_rev_trainer.infer(npr_infer_df, npr_train)


## 4. 
pickup_date2 = pd.to_datetime(pickup_date)+ relativedelta(months=-12)

it = InferenceRevTransformer(duration_of_forecast+4,pickup_date2,basins)
it.fit(raw_df)
infer_df = it.transform(raw_df,oil_df)

# 5. Preprocess CM data
ict = InferenceConMarTrTransformer(rev_data_dict, forecast_start_date)
ict.fit(infer_df)
cmf_infer_df = ict.transform(infer_df)    
cmf_infer_df.to_csv('cmf_infer_df.csv')


# 5.1 

#     icp = InferenceConMarPrTransformer(pr_rev_data_dict,train_basins, forecast_start_date)
#     icp.fit(pr_infer_df)
#     pr_cmf_infer_df = icp.transform(infer_df)

#     icnp = InferenceConMarPrTransformer(npr_rev_data_dict,train_basins,forecast_start_date)
#     icnp.fit(npr_infer_df)
#     npr_cmf_infer_df = icnp.transform(infer_df)
# 5.2
#     npr_cm_model_path = os.path.join(MODEL_PATH, "cm", "NeuralPr")
#     npr_cm_trainer = NeuralProphetModel(npr_features,npr_features,cm_var[0],lagged_features,
#                                          train_basins,pickup_date,npr_cm_model_path)
#     npr_cm_train = npr_cm_trainer.train(npr_train_df)
#     npr_cm_data_dict = npr_cm_trainer.infer(npr_cmf_infer_df,npr_cm_train)
#     npr_cm_data_dict = npr_cm_data_dict.reset_index()[["quarter","SL BASIN (CODE)","cm"]]
#     npr_rev_data_dict = npr_rev_data_dict[["quarter","SL BASIN (CODE)","rev"]]
#     npr_cm_data_dict = npr_cm_data_dict.merge(npr_rev_data_dict[["quarter","SL BASIN (CODE)","rev"]].drop_duplicates(),
#                             on=["quarter","SL BASIN (CODE)"],
#                            how="inner")


def convert_dict_to_df(data_dict):
    baseline_data_dict = pd.DataFrame()
    for basin in data_dict.keys():
        df = data_dict[basin]
        df = df.reset_index()
        df["SL BASIN (CODE)"] = basin
        baseline_data_dict = baseline_data_dict.append(df)
    return baseline_data_dict

### RF Model
cm_model_path = os.path.join(MODEL_PATH, "cm","RF")
cm_trainer = RfModel(cm_features,cm_var,train_basins,pickup_date,cm_model_path)
cm_trainer.train(train_df)
cm_data_dict = cm_trainer.infer(cmf_infer_df)
baseline_data_dict = convert_dict_to_df(cm_data_dict)

#     pr_cm_model_path = os.path.join(MODEL_PATH, "cm","Prophet")
#     pr_cm_trainer = ProphetModel(pr_cm_features,pr_cm_features,cm_var,train_basins,pickup_date,pr_cm_model_path)
#     pr_cm_trainer.train(pr_train_df)
#     pr_cmf_infer_df = pr_cmf_infer_df.reset_index().rename({'quarter':'ds'},axis='columns')
#     pr_cm_data_dict = pr_cm_trainer.infer(pr_cmf_infer_df)
#     pr_cm_data_dict = pr_cm_data_dict.rename({'ds':'quarter'},axis='columns')



# Ground truth
ground_truth = train_df[["SL BASIN (CODE)","cm","rev"]].reset_index().drop_duplicates().dropna()
ground_truth = ground_truth.sort_values(["SL BASIN (CODE)","quarter"])
ground_truth = ground_truth.rename(mapper={"cm":"actual_contribution_margin","rev":"actual_revenue"},axis=1)
ground_truth = ground_truth.reset_index().drop(["index"],axis=1)
ground_truth.to_csv('ground_truth.csv')
# past dataframes
try:
    max_date = pd.read_sql("""SELECT max(refresh_date) FROM cmf3_1_mvp3_Q2""",cxn_ODS)
#         max_date = pd.to_datetime(max_date["max(refresh_date)"][0],format="%Y-%m-%d")
    lr_query = """
    SELECT * FROM cmf3_1_mvp3_Q2 WHERE refresh_date >= '{}' 
    """.format(str(max_date)[-6:])
    multi_model_df = pd.read_sql(lr_query,cxn_ODS)
    multi_model_df = multi_model_df[multi_model_df["quarter"]<forecast_start_date]
    multi_model_df = multi_model_df[["quarter","SL BASIN (CODE)","cm","rev","model"]]

except:
    multi_model_df = pd.DataFrame()
refresh_date = refresh_date.replace(' ', '') 
pp = postprocess(refresh_date, cxn_ODS)
pp.post_process( train_df, baseline_data_dict,"RF", multi_model_df,ground_truth)
#     pp.post_process( train_df, pr_cm_data_dict,"Prophet", multi_model_df,ground_truth)
#     pp.post_process( train_df, npr_cm_data_dict,"Neural Prophet", multi_model_df,ground_truth)



model_sel = model_selection(refresh_date,train_basins)
try:
    best_df = model_sel.best_model(multi_model_df)

    best_df.to_sql(con=cxn_ODS, name='cmf3_1_mvp3_Q2_best',if_exists = 'append',index=None)
except:
    pass


In [ ]:
max_date = pd.read_sql("""SELECT max(refresh_date) FROM cmf3_1_mvp3_Q2""",cxn_ODS)
#         max_date = pd.to_datetime(max_date["max(refresh_date)"][0],format="%Y-%m-%d")
lr_query = """
SELECT * FROM cmf3_1_mvp3_Q2 
"""
multi_model_df = pd.read_sql(lr_query,cxn_ODS)

In [ ]:
multi_model_df.to_csv('multi_model_df.csv')

In [ ]:
try:
    max_date = pd.read_sql("""SELECT max(refresh_date) FROM cmf3_1_mvp3_Q2""",cxn_ODS)
#         max_date = pd.to_datetime(max_date["max(refresh_date)"][0],format="%Y-%m-%d")
    lr_query = """
    SELECT * FROM cmf3_1_mvp3_Q2 WHERE refresh_date >= '{}' 
    """.format(str(max_date)[-6:])
    multi_model_df = pd.read_sql(lr_query,cxn_ODS)
    multi_model_df = multi_model_df[multi_model_df["quarter"]<forecast_start_date]
    multi_model_df = multi_model_df[["quarter","SL BASIN (CODE)","cm","rev","model"]]

except:
    multi_model_df = pd.DataFrame()

In [ ]:
multi_model_df[multi_model_df['SL BASIN (CODE)']=='AML-WEC']

In [ ]:
data_dict = baseline_data_dict.copy()
model = 'RF'


In [ ]:
baseline_data_dict[baseline_data_dict['SL BASIN (CODE)']=='AML-WEC']

In [ ]:
data_dict[data_dict['SL BASIN (CODE)']=='AML-WEC']

In [ ]:
data_dict["model"] = model
columns_out = ["quarter","SL BASIN (CODE)","cm","rev","model"]

multi_model_df = multi_model_df.append(data_dict[columns_out])

multi_model_df = multi_model_df.reset_index().drop(["index"],axis=1)

In [ ]:
multi_model_df[multi_model_df['SL BASIN (CODE)']=='AML-WEC']

In [ ]:
a = multi_model_df.merge(ground_truth,on=["quarter","SL BASIN (CODE)"],how="left")

In [ ]:
a[a['SL BASIN (CODE)']=='AML-WEC']

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Apr 19 12:34:21 2023

@author: BGondaliya
"""

import pandas as pd
import numpy as np
import dateutil.parser as dparser

class postprocess:
    def __init__(self,refresh_date, cxn_ODS):
        self.refresh_date = refresh_date
        self.cxn_ODS = cxn_ODS
        
    def post_process(self, train_df, data_dict, model, multi_model_df, ground_truth):
        data_dict["model"] = model
        columns_out = ["quarter","SL BASIN (CODE)","cm","rev","model"]
        

        multi_model_df = multi_model_df.append(data_dict[columns_out])

        multi_model_df = multi_model_df.reset_index().drop(["index"],axis=1)
       # multi_model_df['quarter'] = pd.to_datetime(multi_model_df['quarter'])
        multi_model_df = multi_model_df.merge(ground_truth,on=["quarter","SL BASIN (CODE)"],how="left")
        multi_model_df["contribution_margin_error"] = np.round(((multi_model_df["cm"]-multi_model_df["actual_contribution_margin"])/multi_model_df["actual_contribution_margin"]),2)*100
        multi_model_df["revenue_error"] = np.round(((multi_model_df["rev"]-multi_model_df["actual_revenue"])/multi_model_df["actual_revenue"]),2)*100
        multi_model_df["YM"] = list(map(lambda x: dparser.parse(str(x),fuzzy=True).strftime('%b\'%y'), multi_model_df["quarter"]))
        multi_model_df["refresh_date"] = self.refresh_date
        multi_model_df["level"] = multi_model_df["SL BASIN (CODE)"]
        multi_model_df = multi_model_df.round(2) 


        multi_model_df[multi_model_df["SL BASIN (CODE)"]=="Worldwide"].sort_values(["SL BASIN (CODE)","quarter","model"]).dropna()
        multi_model_df = multi_model_df.drop_duplicates()
        multi_model_df.to_sql(con=self.cxn_ODS, name='cmf3_1_mvp3_Q2',if_exists = 'append',index=None)
       # print(" Written for Model - {}".format(, model))



In [ ]:
refresh_date = refresh_date.replace(' ', '') 
pp = postprocess(refresh_date, cxn_ODS)
pp.post_process( train_df, baseline_data_dict,"RF", multi_model_df,ground_truth)
#     pp.post_process( train_df, pr_cm_data_dict,"Prophet", multi_model_df,ground_truth)
#     pp.post_process( train_df, npr_cm_data_dict,"Neural Prophet", multi_model_df,ground_truth)



model_sel = model_selection(refresh_date,train_basins)
try:
    best_df = model_sel.best_model(multi_model_df)

    best_df.to_sql(con=cxn_ODS, name='cmf3_1_mvp3_Q2_best',if_exists = 'append',index=None)
except:
    pass


In [ ]:
ref_date = refresh_date
refresh_date = str(pd.to_datetime(refresh_date).replace(day=1)) 

cutoff_date = pd.to_datetime(refresh_date) + relativedelta(months=-12)
cutoff_date = cutoff_date.strftime('%YQ{}'.format( (cutoff_date.month - 1) // 3 + 1))
train_basins = train_basins


In [ ]:
try:
    max_date = pd.read_sql("""SELECT min(refresh_date) FROM cmf3_1_mvp3_Q2""",cxn_ODS)
#         max_date = pd.to_datetime(max_date["max(refresh_date)"][0],format="%Y-%m-%d")
    lr_query = """
    SELECT * FROM cmf3_1_mvp3_Q2 WHERE refresh_date <= '{}' 
    """.format(str(max_date)[-6:])
    multi_model_df = pd.read_sql(lr_query,cxn_ODS)
#     multi_model_df = multi_model_df[multi_model_df["quarter"]<forecast_start_date]
    multi_model_df = multi_model_df[["quarter","SL BASIN (CODE)","cm","rev","model"]]

except:
    multi_model_df = pd.DataFrame()

In [ ]:
multi_model_df

In [ ]:
cutoff_date

In [ ]:
self.cutoff_date = pd.to_datetime(cutoff_date, format="%Y-%m-%d")
self.cutoff_date = cutoff_date.strftime('%Y Q{}'.format( (cutoff_date.month - 1) // 3 + 1))

In [ ]:
refresh_date

In [ ]:
ref_date

In [ ]:
cutoff_date

In [ ]:
X = multi_model_df.copy()

In [ ]:
X

In [ ]:
# data = X[X['refresh_date']== ref_date]
data = X[(X['quarter'] > cutoff_date) & (X['quarter'] < refresh_date)]
data

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  6 11:17:50 2023

@author: BGondaliya
"""

import datetime
from datetime import date
import pandas as pd
from dateutil.relativedelta import relativedelta


class model_selection:
    def __init__(self,refresh_date,train_basins):
        self.ref_date = refresh_date
        self.refresh_date = str(pd.to_datetime(refresh_date).replace(day=1)) #str(date.today(
        self.cutoff_date = str(pd.to_datetime(refresh_date) + relativedelta(months=-4))
        self.train_basins = train_basins

    
    def best_model(self, X):
        data = X[X['refresh_date']== self.ref_date]
        data = data[(data['quarter'] > self.cutoff_date) & (data['quarter'] < self.ref_date)]

        data['abs_rev_error'] = abs(data['revenue_error'])
        data['abs_cm_error'] = abs(data['contribution_margin_error'])
        
        grouped_df = data.groupby(['SL BASIN (CODE)', 'model','refresh_date']).agg({'abs_rev_error': 'mean', 'abs_cm_error': 'mean'})
        grouped_df = grouped_df.dropna()
        
        grouped = grouped_df.groupby('SL BASIN (CODE)')
        min_indices = grouped['abs_rev_error'].idxmin()
        best_rev_df = grouped_df.loc[min_indices]
        
        grouped = grouped_df.groupby('SL BASIN (CODE)')

        min_indices = grouped['abs_cm_error'].idxmin()
        best_cm_df = grouped_df.loc[min_indices]
        
        # to extract dataframe for best model
        data = X[X['refresh_date']== self.ref_date]
#         data = data[data['refresh_date']==self.refresh_date]
        ans = pd.DataFrame(columns=['YEARMONTH', 'SL BASIN (CODE)', 'rev', 'actual_revenue',
               'revenue_error', 'YM', 'refresh_date', 'level', 'rev_Model', 'cm',
               'actual_contribution_margin', 'contribution_margin_error', 'cm_Model'])   
        for basin in self.train_basins:
            print(basin)

            rev_model = best_rev_df.loc[best_rev_df.index.get_level_values('SL BASIN (CODE)') == basin].index.get_level_values('model')
            cm_model = best_cm_df.loc[best_cm_df.index.get_level_values('SL BASIN (CODE)') == basin].index.get_level_values('model')

            df_rev = data[(data['SL BASIN (CODE)']==basin) &(data['model']=="RF")][['YEARMONTH', 'SL BASIN (CODE)', 'rev',
                    'actual_revenue','revenue_error', 'YM', 'refresh_date',
                   'level']]
            df_rev['rev_Model'] = str(rev_model[0])
            df_rev = df_rev.drop_duplicates()

            df_cm = data[(data['SL BASIN (CODE)']==basin) &(data['model']=="RF")][['YEARMONTH', 'SL BASIN (CODE)', 'cm',
                   'actual_contribution_margin', 
                   'contribution_margin_error', ]]
            df_cm['cm_Model'] = str(cm_model[0])
            df_cm = df_cm.drop_duplicates()

            ans = ans.append(pd.merge(df_rev,df_cm, how='left', on = ['YEARMONTH','SL BASIN (CODE)']))

        return ans